In [1]:
#Load important and necessary libraries 

from googleapiclient.discovery import build
import pandas as pd
from IPython.display import JSON
import calendar
%matplotlib inline

In [15]:
channel_ids = ['UC7cs8q-gJRlGwj4A8OmCmXg']   
#['UCvZnwzmc3m1Eush-Or8Z6DA','UC7cs8q-gJRlGwj4A8OmCmXg','UC0p5jTq6Xx_DosDFxVXnWaQ','UCZ4AMrDcNrfy3X6nsU8-rPg']   
#List of channel id's we want to grab and analyze  Alex_the_Analyst, Shashank, The_Economist, Econ_Expalined

In [16]:
#Setup Youtube API call and specifications
api_service_name = "youtube"
api_version = "v3"

# Get credentials and create an API initialization
youtube = build(api_service_name, api_version, developerKey=api_key)

In [17]:
def pull_info(youtube, channel_ids): #function that takes in youtube API call and list of channels you want to pull data for and returns a dataframe
    
    data = [] 
    
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_ids)
    )
    response = request.execute()
    
    #create loop to grab channel statistics
    for item in response['items']:
        stats = {'channel_name': item['snippet']['title'],
                 'subscribers': item['statistics']['subscriberCount'],
                 'views': item['statistics']['viewCount'],
                 'total_videos': item['statistics']['videoCount'],
                 'playlist_id': item['contentDetails']['relatedPlaylists']['uploads']
                }
        data.append(stats)
    return(pd.DataFrame(data))

In [18]:
pull_info(youtube, channel_ids)

,channel_name,subscribers,views,total_videos,playlist_id
0,Alex The Analyst,221000,8678776,144,UU7cs8q-gJRlGwj4A8OmCmXg


In [19]:
# Take each playlist id in pull_info
playlist_id_group = [i for i in pull_info(youtube, channel_ids)['playlist_id']]
playlist_id_group

['UU7cs8q-gJRlGwj4A8OmCmXg']

In [20]:
playlist_id = ['UUvZnwzmc3m1Eush-Or8Z6DA'] #'UUvZnwzmc3m1Eush-Or8Z6DA','UU7cs8q-gJRlGwj4A8OmCmXg'

def video_ids(youtube, playlist_id): #take the playlist id and return all of the video id's that are on this playlist
    
    vid_ids = [] #create empty list for video id's
    
    request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId=','.join(playlist_id),
            maxResults=50
        )
    response = request.execute() #call the api to get the most recent 50 videos on this playlist
    
    for item in response['items']:
        vid_ids.append(item['contentDetails']['videoId']) #look at the content details of the video and grab the unique id for it
    
#     next_page = response.get('nextPageToken')
#     while next_page:
#         request = youtube.playlistItems().list(
#             part='contentDetails',
#             playlistId = playlist_id,
#             maxResults=50,
#             pageToken= next_page)
#         response = request.execute()
        
#         for item in response['items']:
#             vid_ids.append(item['contentDetails']['videoId'])
        
#         next_page = response.get('nextPageToken')
        
    return vid_ids

In [21]:
video_ids = video_ids(youtube,playlist_id) #create list of video ids that were pulled (50) is max that youtube allows
len(video_ids)

50

In [22]:
def video_details(youtube, video_id): #take each video id and get specified metrics
    
    all_details = []
    
    for i in range(0, len(video_ids),50):
        request = youtube.videos().list(
                part="snippet,contentDetails,statistics",
                id=','.join(video_ids[i:i+50])
            )
        response = request.execute()


        for video in response['items']:
            stats_pulled = {'snippet':['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                            'statistics': ['viewCount', 'likeCount', 'commentCount'],
                            'contentDetails': ['duration']
                           }
            video_info = {}
            video_info['video_id'] = video['id']

            for h in stats_pulled.keys():
                for j in stats_pulled[h]:
                    try:
                        video_info[j] = video[h][j]
                    except:
                        video_info[h] = None
            all_details.append(video_info)
            
    return pd.DataFrame(all_details) #return the requested data in a dataframe for analysis

In [23]:
video_df = video_details(youtube,video_ids) #crete dataframe with all 50 vidoes and all video metrics
video_df.shape

(50, 10)

In [ ]:
#Convert the youtube string columns into numeric using apply mehthod
to_num_cols = ['viewCount','likeCount','commentCount']
video_df[to_num_cols] = video_df[to_num_cols].apply(pd.to_numeric, errors='coerce', axis=1)

In [ ]:
#Get day of week video was published
import datetime
video_df['datePublished'] = video_df['publishedAt'].str[:10]
video_df['dayofMonth'] = video_df['datePublished'].str[-2:]
video_df['datePublished'] = video_df['datePublished'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))

In [ ]:
#Convert Youtube API video length to seconds
import isodate
video_df['durationSeconds'] = video_df['duration'].apply(lambda x: isodate.parse_duration(x)).astype('timedelta64[s]')

# Create Excel file

In [24]:
video_df.to_excel('AtA_youtube.xlsx')